In [114]:
import pandas as pd
import re, os, json
from dotenv import load_dotenv
import google.generativeai as genai
import textwrap, io
from IPython.display import Markdown
from IPython.display import display
from sklearn.metrics import confusion_matrix

In [2]:
amazon_alexa = pd.read_csv('amazon_alexa.tsv', delimiter="\t")
df =amazon_alexa[['verified_reviews', 'feedback']]
df.columns= ['reviews', 'feedback']

In [3]:
df.head(10)

,reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1
5,I received the echo as a gift. I needed anothe...,1
6,"Without having a cellphone, I cannot use many ...",1
7,I think this is the 5th one I've purchased. I'...,1
8,looks great,1
9,Love it! I’ve listened to songs I haven’t hear...,1


In [4]:
rows_to_drop = df['feedback'].value_counts().max() - df['feedback'].value_counts().min()

In [5]:
if rows_to_drop > 0:
    df_1 = df[df['feedback'] == 1]
    df_balanced = df.drop(df_1.sample(rows_to_drop).index)
 

In [6]:
df_balanced.feedback.value_counts()


feedback
1    257
0    257
Name: count, dtype: int64

In [7]:
""" \w matches with words a-zA-Z, 0-1 and _(underscore)
\s gives only space removing all the word
inthis case \w gives words after removing space
"""
word ="1. I am Ajeet and this is Me Ajeet!" 

In [8]:
def input_format(word):
    word = re.sub(r"[^\w\s]", "", word)
    word = re.sub(r"\b[a-zA-Z]\b", " ", word)        
    word = word.lower()
    word = re.sub(r"\s+"," " ,word)
    word = re.sub(r"<[^>]*>", " ", word)
    word = word.strip()
    return word

In [9]:
word = re.sub(r"[^\w\s]", "", word)
word

'1 I am Ajeet and this is Me Ajeet'

In [10]:
word = re.sub(r"\b[a-zA-Z]\b", " ", word)
word

'1   am Ajeet and this is Me Ajeet'

In [11]:
word = word.lower()
word

'1   am ajeet and this is me ajeet'

In [12]:
word = re.sub(r"\s+"," " ,word)
word

'1 am ajeet and this is me ajeet'

In [13]:
word = re.sub(r"<[^>]*>", " ", word)

In [14]:
word = word.strip()
word

'1 am ajeet and this is me ajeet'

In [15]:
df_balanced['reviews']

9       Love it! I’ve listened to songs I haven’t hear...
19      I liked the original Echo. This is the same bu...
31                                              I like it
32      She works well. Needs a learning command  for ...
38      This thing is way cool!  You should get one.  ...
                              ...                        
3082    Easy to set up and start using.. works as adve...
3084    Love our little dot. I don’t know what the ori...
3091                                    I didn’t order it
3096    The product sounded the same as the emoji spea...
3124    Love my Alexa! Actually have 3 throughout the ...
Name: reviews, Length: 514, dtype: object

In [16]:
df_balanced['reviews'] = df_balanced['reviews'].astype(str)

In [17]:
reviews = df_balanced['reviews'].to_list()

In [18]:
[input_format(review) for review in reviews]


['love it ive listened to songs havent heard since childhood get the news weather information its great',
 'liked the original echo this is the same but shorter and with greater fabriccolor choices miss the volume ring on top now its just the plusminus buttons not big deal but the ring as comforting other than that well do like the use of standard usb charger port instead of the previous round pin other than that guess it sounds the same seems to work the same still answers to alexaechocomputer so whats not to like',
 'like it',
 'she works well needs learning command for unique owners and users like alexa learn tashas birthday or alexa learn my definition of fine etc other than that she is great',
 'this thing is way cool you should get one if you want to be cool that is',
 'tried to play certain broadway shows like camelot and it gives ne the group camelot',
 'its like siri in fact siri answers more accurately then alexa dont see real need for it in my household though it was good ba

In [19]:
df_balanced['reviews'] = [input_format(review) for review in reviews]
df_balanced

,reviews,feedback
9,love it ive listened to songs havent heard sin...,1
19,liked the original echo this is the same but s...,1
31,like it,1
32,she works well needs learning command for uniq...,1
38,this thing is way cool you should get one if y...,1
...,...,...
3082,easy to set up and start using works as advert...,1
3084,love our little dot dont know what the origina...,1
3091,didnt order it,0
3096,the product sounded the same as the emoji spea...,0


In [20]:
len(df_balanced)
test_size = int(len(df_balanced) * 0.90)
train_df = df_balanced.sample(test_size)
test_df = df_balanced.drop(train_df.index)

In [21]:
train_df

,reviews,feedback
588,works great to control the tv lights and vario...,1
1267,switched to google the amazon helper is more f...,0
2892,cheap and cheap sound,0
1447,,0
2019,,0
...,...,...
1145,didnt think id get an alexa device but this is...,1
1571,all of my echo devices stopped communicating p...,0
2798,good,1
951,have the original alexa and the tap and so far...,1


In [22]:
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [23]:
# def to_markdown(text):


In [24]:
def to_markdown(text):
    text = text.replace('', '')
    return Markdown(textwrap.indent(text, '>' , predicate=lambda _: True))

In [25]:
model = genai.GenerativeModel("gemini-pro")
response = model.generate_content('What is guitar?')
to_markdown(response.text)

>Life is a complex and multifaceted phenomenon that can be defined in many ways. Here are some common definitions:
>
>**Biological Definition:**
>Life is a self-sustaining, self-organizing system capable of performing the functions of metabolism, growth, and reproduction. This definition emphasizes the physical and chemical processes that underlie life.
>
>**Philosophical Definition:**
>Life is a state of being that possesses consciousness, intentionality, and the ability to experience the world. This definition focuses on the subjective and qualitative aspects of life.
>
>**Scientific Definition:**
>Life is a process that sustains and reproduces itself through the exchange of energy and matter with its environment. This definition emphasizes the dynamic nature of life and its interconnectedness with the surrounding ecosystem.
>
>**Theological Definition:**
>Life is a gift from a higher power or God. This definition emphasizes the spiritual or religious aspects of life and its ultimate purpose.
>
>**Emergent Definition:**
>Life is a complex phenomenon that emerges from the interaction of simpler components, such as molecules or cells. This definition emphasizes the hierarchical nature of life and its potential for self-organization.
>
>**Anthropic Definition:**
>Life is anything that is capable of perceiving and interacting with its environment. This definition is based on the idea that life is a subjective experience and that our understanding of it is limited by our own human perspective.
>
>Ultimately, there is no single, definitive definition of life. The concept is vast and encompasses both physical and philosophical aspects. The definition of life continues to be debated and refined by scientists, philosophers, and theologians.

In [26]:
df_balanced

,reviews,feedback
9,love it ive listened to songs havent heard sin...,1
19,liked the original echo this is the same but s...,1
31,like it,1
32,she works well needs learning command for uniq...,1
38,this thing is way cool you should get one if y...,1
...,...,...
3082,easy to set up and start using works as advert...,1
3084,love our little dot dont know what the origina...,1
3091,didnt order it,0
3096,the product sounded the same as the emoji spea...,0


## Zero shot Sentiment Classification

In [27]:
df_balanced['pred'] = ''
df_json_zero= df_balanced[['reviews', 'pred', 'feedback']].sample(20).to_json(orient='records')

In [77]:

prompt_zero = f""" You are an expert linguist, who is good at classifying review Sentiment into Positive/Negative labels. 
Help me classify customer reviews into Positive(label = 1) or Negative(label = 0) Customer reviews are provided between three back ticks.
In your output, only return the json code back as output- which is provided between three back ticks.
Your task is to update predicted labels under 'pred_labels' in the json code.
Don't make any changes to the Json code format, please
```
{df_json_zero}
```
"""

In [29]:
response_zero = model.generate_content(prompt_zero)

In [176]:
response_zero.text

'```\n[{"reviews":"love it use it for everything its part of my daily routine","pred":1,"feedback":1},{"reviews":"its like siri in fact siri answers more accurately then alexa dont see real need for it in my household though it was good bargain on prime day deals","pred":0,"feedback":0},{"reviews":"bought an echo dot that had been refurbished but its stops whatever function its performing quite often doesnt wake up every time randomly turns on but does nothing do not buy refurbished echo dot","pred":0,"feedback":0},{"reviews":"it is very slow compared to the echo","pred":0,"feedback":0},{"reviews":"works on great use it with my arlo security system","pred":1,"feedback":1},{"reviews":"great fun getting to know all the functions of this product wow family fun and homework help talking with other grandchildrenwho also have an echo is huge bonus cant wait to learn more and more and more","pred":1,"feedback":1},{"reviews":"item returned for repair receivded item back from repair 072318 part

In [30]:
response_data_zero = pd.DataFrame(json.loads(response_zero.text.strip("`")))
response_data_zero

,reviews,pred,feedback
0,love it use it for everything its part of my d...,1,1
1,its like siri in fact siri answers more accura...,0,0
2,bought an echo dot that had been refurbished b...,0,0
3,it is very slow compared to the echo,0,0
4,works on great use it with my arlo security sy...,1,1
5,great fun getting to know all the functions of...,1,1
6,item returned for repair receivded item back f...,0,0
7,doesnt always respond when spoken to with prom...,0,0
8,works wonderfully,1,1
9,love every aspect of the echo plus wanted it f...,1,1


In [44]:
y = response_data_zero['feedback']
y_hat = response_data_zero['pred']
confusion_matrix(y,y_hat)

array([[10,  0],
       [ 0, 10]])

## Few shot sentiment classification

In [45]:
df_balanced

,reviews,feedback,pred
9,love it ive listened to songs havent heard sin...,1,
19,liked the original echo this is the same but s...,1,
31,like it,1,
32,she works well needs learning command for uniq...,1,
38,this thing is way cool you should get one if y...,1,
...,...,...,...
3082,easy to set up and start using works as advert...,1,
3084,love our little dot dont know what the origina...,1,
3091,didnt order it,0,
3096,the product sounded the same as the emoji spea...,0,


In [282]:
few_shot = test_df.sample(20)
few_shot_1 = few_shot.to_json() # this is to give to the model for few shot


In [303]:
df_json_few = df_balanced[['reviews','feedback']].sample(20)
df_json_few_1 = df_json_few
df_json_few_1['feedback'] = '' # this is what the model will predict
df_json_few_1 = df_json_few.to_json()


,reviews,feedback
1540,fire hd does so much more over priced pair of ...,
620,brought to replace one that moved and doesnt h...,
1612,great device features are awesome the interact...,
1366,love it,
819,it was very easy to set up and my family love its,
623,love it,
2383,garbageeven trying to watch amazon prime video...,
2257,works wonderfully,
3060,easy setup easy use work with other smart prod...,
1352,the product is attractive and somewhat useful ...,


You are an expert linguist, who is good at classifying review Sentiment into Positive/Negative labels. 
Help me classify customer reviews into Positive(label = 1) or Negative(label = 0) Customer reviews are provided between three back ticks.
In your output, only return the json code back as output- which is provided between three back ticks.
Your task is to update predicted labels under 'pred_labels' in the json code.
Don't make any changes to the Json code format, please









You are a sentiment analysis expert who has just received a few examples of positive and negative customer reviews. You need to classify the sentiment of additional customer reviews based on this limited training data.

Below are a few examples of positive and negative reviews:


In [289]:

# # prompt_few =  f"""
# # You are an expert linguist who is good at classifying review Sentiments into Positive(label = 1) and negative labels(label = 0). 
# # You have just received a few examples of Positive(1) and Negative(0) customer reviews which is given below between three back ticks:
# # {prompt_example}
# # Help me classify the sentiment of additional customer based on this limited training data. The data you have to classify are provided 
# # between three back ticks.  In your output, only return the json code back as output- which is provided between three back ticks. 
# # Your task is to update predicted labels under 'pred_labels' in the json code. Don't make any changes to the json code format,please.

# # `{df_json_few}`
# # """


In [290]:

prompt_few = f""" You are an expert linguist, who is good at classifying review Sentiment into Positive/Negative labels. 
You have just received a few examples of Positive(1) and Negative(0) customer reviews which is given below between one back ticks:
`{few_shot_1}`
Help me classify customer reviews into Positive(label = 1) or Negative(label = 0) Customer reviews are provided between three back ticks.
In your output, only return the json code back as output- which is provided between three back ticks.
Your task is to update predicted labels under 'feedback' in the json code without a json tag.
Don't make any changes to the Json code format, please.
```
{df_json_few_1}
```
"""

In [291]:
response_few = model.generate_content(prompt_few)

In [292]:

response_data_few_shot = pd.DataFrame(json.loads(response_few.text.strip("`")))
response_data_few_shot


,reviews,feedback
839,everyone should have an echo or two or three o...,1
1122,love being able to have alexa sing careless wh...,1
2305,had to return it as hulu does not work on devi...,0
2013,bought this for myself and didnt realize it ha...,0
280,love it have one inside house and one by our p...,1
1796,alexa didnt know answers to many questions we ...,0
2074,didnt really work as smooth as they make it se...,0
1951,the echo part works fine the hue hub is disast...,0
1783,like the whole thing so far,1
2581,this device does not interact with my home fil...,0


In [297]:
y = df_json_few['feedback']
y_pred = response_data_few_shot['feedback']
confusion_matrix(y,y_pred)

array([[11,  0],
       [ 0,  9]])